### Forward and Simulation
* Read forward solution
* Read stored simulation

In [ ]:
from forward import solve_forward
from simulation import Simulation
import numpy as np

In [ ]:
fwd = solve_forward()

# load eeg and source data from previous simulations
eeg_data = np.load('/media/thanos/Big Data/Thanos/TUC/Thesis/sim_data/eeg_big_sim_2.npy')
source_data = np.load('/media/thanos/Big Data/Thanos/TUC/Thesis/sim_data/sources_big_sim_2.npy')
sim = Simulation(fwd=fwd,source_data=source_data,eeg_data=eeg_data)

### Neural Network
A simple neural network with only one hidden dense layer.

In [ ]:
from net import EEGNet

eegnet = EEGNet(fwd=fwd,sim=sim)

In [ ]:
eegnet.build_model()

In [ ]:
eegnet.fit()